In [ ]:
import os
import glob
import re
import pandas as pd 
import shutil
import re
import math

In [ ]:
fpath="Acoustic_Chambers_Calibration_Data/CalibrationSetupFile.csv"
folder_path = os.path.dirname(os.path.dirname(fpath))

calib_setup = pd.read_csv(fpath,header=None)

column_list=list(calib_setup.loc[calib_setup[0]==1000].iloc[0,:])
calib_setup.columns=column_list

column_list =  [x for x in column_list if not math.isnan(x)]
column_list=[round(num) for num in column_list]
column_list=column_list[1::]

In [ ]:
ACC_setup_file_Path = ("Acoustic_Chambers_Calibration_Data/CalibrationSetupFile.csv")

dir_path = os.path.dirname(ACC_setup_file_Path)

# Generate the new ACC folder which duplitude all contents
Old_ACC_Folder = "Acoustic_Chambers_Calibration_Data"
New_ACC_Folder=shutil.copytree(Old_ACC_Folder, Old_ACC_Folder+'_new')
New_ACC_Path = os.path.join(os.path.dirname(Old_ACC_Folder), 'Acoustic_Chambers_Calibration_Data_New')

In [ ]:
regex = re.compile(".*CalibSpeakersVoltageRMS.*")   # search the file in ACC folder (Speaker voltage file) 
regex1 = re.compile(".*SystemPhase_94dBSPL.*")      # search the file in ACC folder (SystemPhase file) 

writer1 = pd.ExcelWriter('output_spk.xlsx', engine='xlsxwriter')
writer2 = pd.ExcelWriter('output_Phase.xlsx', engine='xlsxwriter')

# get the each value in column_list, using glob module to find the folder which including this value 
for value in column_list:
    search_pattern = os.path.join(New_ACC_Folder, f"*{value}*")
    folders = glob.glob(search_pattern)
    if len(folders) > 0:
        for folder in folders:
            print(f"Found folder for value {value}: {folder}")
            
            # search the file from the folderwhich get from last step.
            files = glob.glob(os.path.join(folder, "*.csv"))
            if len(files) > 0:
                for file in files:
                    # find the file including "CalibSpeakersVoltageRMS"
                    match = regex.search(file)
                    if match and 'Real' not in file:
                        
                        print(f"Found file: {file}")

                        data=pd.read_csv(file)
                        sheet_name = str(value)

                        data.to_excel(writer1,sheet_name=sheet_name,index=False) 

                for file in files:
                    match = regex1.search(file)
                    if match:
                        print(f"Found file: {file}")
                        data = pd.read_csv(file)
                        sheet_name = str(value)

                        data.to_excel(writer2,sheet_name=sheet_name,index=False) 
            else:
                print(f"No files found in folder {folder}")
    else:
        print(f"No folders found for value {value}")
        
writer1.close()
writer2.close()

In [ ]:
#Phase compensation and ACC file system phase compensation and saved

folder_path = "output_Phase.xlsx"
data = pd.read_excel(folder_path, sheet_name=None)
# print(data)

AMSA2_Path = 'AMSA2.xlsx'
AMSA2 = pd.read_excel(AMSA2_Path, sheet_name=None)

if '94' in data.keys():
    data_94 = data['94'] 
    print(data_94)

    for sheet_name, sheet_data in AMSA2.items():
        if 'Phase' in sheet_name:
            match = re.search(r'\d+', sheet_name)
            if match:
                numeric_part = match.group()
                
                if numeric_part in data_94.columns.values:
                    data_94_column = data_94[numeric_part]

                    # add one column in AMSA2， add Data_94_column in this new column
                    sheet_data['SystemPhase'] = data_94_column
                    sheet_data['deltaPhase']=sheet_data['PHASE']-sheet_data['Phase_New_Target']
                    sheet_data['New_SystemPhase']=sheet_data['SystemPhase']+sheet_data['deltaPhase']
                    sheet_data[numeric_part]=sheet_data['New_SystemPhase']
                    
                    #save the new spk voltage in the ACC foler - 94dBspl speaker vol
                    for root, dirs, files in os.walk(New_ACC_Path):
                        for file in files:

                            #94dBspl speaker vol updated in ACC and saved in the new ACC folder 
                            if file=='SystemPhase_94dBSPL.csv':
                                Spk94DBSPL_file_path=os.path.join(root,file)
                                Spk94DBSPL_data=pd.read_csv(Spk94DBSPL_file_path)

                                if numeric_part in Spk94DBSPL_data.columns:
                                    # Spk94DBSPL_data.loc[:, sheet_name] = sheet_data['vol_ratio']*sheet_data['spkVol']
                                    Spk94DBSPL_data.loc[:, numeric_part] = sheet_data[numeric_part]
                                    Spk94DBSPL_data.to_csv(Spk94DBSPL_file_path, index=False)
                                else:
                                    continue

with pd.ExcelWriter(AMSA2_Path, engine='openpyxl') as writer:
    for sheet_name, sheet_data in AMSA2.items():
        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)           

In [ ]:
#amplitude compensation and ACC file Speaker voltage / RefMic MIC Amplitude compensation and saved

folder_path_1 = "output_spk.xlsx"
data_1 = pd.read_excel(folder_path_1, sheet_name=None)

AMSA2_Path = 'AMSA2.xlsx'
AMSA2 = pd.read_excel(AMSA2_Path, sheet_name=None)

if '94' in data_1.keys():
    data_94_1 = data_1['94'] 
    print(data_94_1)


    for sheet_name, sheet_data in AMSA2.items():
        if sheet_name in data_94_1.columns.values:            
            # from data_94 get the corrlated column data.
            data_94_1_column = data_94_1[sheet_name]
        
            
            # 在AMSA2，sheet add one new column， add data_94_column to new column
            sheet_data['spkVol'] = data_94_1_column
            sheet_data['deltaSens']=sheet_data['New_Target']-sheet_data['SENS']
            sheet_data['vol_ratio']=10**(sheet_data['deltaSens']/20)
            sheet_data['New_spkVol']=sheet_data['vol_ratio']*sheet_data['spkVol']
            sheet_data[sheet_name]=sheet_data['vol_ratio']*sheet_data['spkVol']

            
            #generate the new speaker vol, save this voltage to new ACC folder - 94dBspl speaker vol file
            for root, dirs, files in os.walk(New_ACC_Path):
                for file in files:
                    #94dBspl speaker vol updated in ACC and saved in the new ACC folder 
                    if file=='CalibSpeakersVoltageRMS_94dBSPL.csv':
                        Spk94DBSPL_file_path=os.path.join(root,file)
                        Spk94DBSPL_data=pd.read_csv(Spk94DBSPL_file_path)

                        if sheet_name in Spk94DBSPL_data.columns:
                            Spk94DBSPL_data.loc[:, sheet_name] = sheet_data[sheet_name]
                            Spk94DBSPL_data.to_csv(Spk94DBSPL_file_path, index=False)

                    #94dBspl ref mic output - amplitude value updated and saved in new ACC folder
                    if file=='ReferenceMicAmplitude_94dBSPL.csv':
                        Ref_output_file=os.path.join(root,file)
                        Ref_output_data=pd.read_csv(Ref_output_file)
                    
                        if sheet_name in Ref_output_data.columns:

                            Ref_output_data.loc[:,sheet_name]=Ref_output_data.loc[:,sheet_name]+sheet_data['deltaSens']
                            Ref_output_data.to_csv(Ref_output_file,index=False)

#save all data in AMSA2
with pd.ExcelWriter(AMSA2_Path, engine='openpyxl') as writer:
    for sheet_name, sheet_data in AMSA2.items():
        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
# THD Compensation 

AMSA2_Path = 'AMSA2.xlsx'
AMSA2 = pd.read_excel(AMSA2_Path, sheet_name=None)


for sheet_name, sheet_data in AMSA2.items():
    if sheet_name=='1000':
        data_94_THD = sheet_data['New_spkVol']

for sheet_name, sheet_data in AMSA2.items():

    if 'THD' in sheet_name:
        match = re.search(r'\d+', sheet_name)
        if match:
            numeric_part = int(match.group())

            sheet_data['94dBspl'] = data_94_THD
            sheet_data['Delta_SPL'] = numeric_part - 94
            sheet_data['Sens']=10**(sheet_data['Delta_SPL']/20)*sheet_data['94dBspl']
            sheet_data[numeric_part]=sheet_data['Sens']

            for root, dirs, files in os.walk(New_ACC_Path):
                for file in files:

                    #94dBspl speaker vol updated in ACC and saved in the new ACC folder 
                    if file=='CalibSpeakersVoltageRMS_'+str(numeric_part)+'dBSPL.csv' and file!='CalibSpeakersVoltageRMS_94dBSPL.csv' : 
                        # print(file)
                        Spk94DBSPL_file_path=os.path.join(root,file)
                        Spk94DBSPL_data=pd.read_csv(Spk94DBSPL_file_path)

                        if '1000' in Spk94DBSPL_data.columns:
                            # Spk94DBSPL_data.loc[:, sheet_name] = sheet_data['vol_ratio']*sheet_data['spkVol']
                            Spk94DBSPL_data.loc[:, '1000'] = sheet_data[numeric_part]
                            Spk94DBSPL_data.to_csv(Spk94DBSPL_file_path, index=False)
                            
with pd.ExcelWriter(AMSA2_Path, engine='openpyxl') as writer:
    for sheet_name, sheet_data in AMSA2.items():
        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)    